In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, log_loss
pd.options.display.precision = 15
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
import time
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import json
%matplotlib inline

%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [49]:
train = pd.read_csv('./datab/original_train.csv')
testa = pd.read_csv('./datab/original_test_A.csv')
testb = pd.read_csv('./datab/original_test_B.csv')
print(train.shape,testa.shape,testb.shape)

(40000, 51) (12000, 50) (12000, 50)


In [50]:
import copy
test = copy.deepcopy(testb)
train['TARGET'] = 0
test['TARGET'] = 1

In [51]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
def fast_auc(y_true, y_prob):
    """
    fast roc_auc computation: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/76013
    """
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc


def eval_auc(y_true, y_pred):
    """
    Fast auc eval function for lgb.
    """
    return 'auc', fast_auc(y_true, y_pred), True


def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()
    

def train_model_regression(X, X_test, y, params, folds=None, model_type='lgb', eval_metric='mae', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3):
    """
    A function to train a variety of regression models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    X_test = X_test[columns]
    splits = folds.split(X) if splits is None else splits
    n_splits = folds.n_splits if splits is None else n_folds
    
    # to set up scoring parameters
    metrics_dict = {'mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'sklearn_scoring_function': metrics.mean_absolute_error},
                    'group_mae': {'lgb_metric_name': 'mae',
                        'catboost_metric_name': 'MAE',
                        'scoring_function': group_mean_log_mae},
                    'mse': {'lgb_metric_name': 'mse',
                        'catboost_metric_name': 'MSE',
                        'sklearn_scoring_function': metrics.mean_squared_error}
                    }

    
    result_dict = {}
    
    # out-of-fold predictions on train data
    oof = np.zeros(len(X))
    
    # averaged predictions on train data
    prediction = np.zeros(len(X_test))
    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(splits):
        if verbose:
            print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = n_estimators, n_jobs = -1)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict(X_test).reshape(-1,)
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function=metrics_dict[eval_metric]['catboost_metric_name'])
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        if eval_metric != 'group_mae':
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
        else:
            scores.append(metrics_dict[eval_metric]['scoring_function'](y_valid, y_pred_valid, X_valid['type']))

        prediction += y_pred    
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12))
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)')
            
            result_dict['feature_importance'] = feature_importance
        
    return result_dict
    


def train_model_classification(X, X_test, y, params, folds, model_type='lgb', eval_metric='auc', columns=None, plot_feature_importance=False, model=None,
                               verbose=10000, early_stopping_rounds=200, n_estimators=50000, splits=None, n_folds=3, averaging='usual', n_jobs=-1):
    """
    A function to train a variety of classification models.
    Returns dictionary with oof predictions, test predictions, scores and, if necessary, feature importances.
    
    :params: X - training data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: X_test - test data, can be pd.DataFrame or np.ndarray (after normalizing)
    :params: y - target
    :params: folds - folds to split data
    :params: model_type - type of model to use
    :params: eval_metric - metric to use
    :params: columns - columns to use. If None - use all columns
    :params: plot_feature_importance - whether to plot feature importance of LGB
    :params: model - sklearn model, works only for "sklearn" model type
    
    """
    columns = X.columns if columns is None else columns
    n_splits = folds.n_splits if splits is None else n_folds
    X_test = X_test[columns]
    
    # to set up scoring parameters
    metrics_dict = {'auc': {'lgb_metric_name': eval_auc,
                        'catboost_metric_name': 'AUC',
                        'sklearn_scoring_function': metrics.roc_auc_score},
                    }
    
    result_dict = {}
    if averaging == 'usual':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))
        
    elif averaging == 'rank':
        # out-of-fold predictions on train data
        oof = np.zeros((len(X), 1))

        # averaged predictions on train data
        prediction = np.zeros((len(X_test), 1))

    
    # list of scores on folds
    scores = []
    feature_importance = pd.DataFrame()
    
    # split and train on folds
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print(f'Fold {fold_n + 1} started at {time.ctime()}')
        if type(X) == np.ndarray:
            X_train, X_valid = X[columns][train_index], X[columns][valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
        else:
            X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
            
        if model_type == 'lgb':
            model = lgb.LGBMClassifier(**params, n_estimators=n_estimators, n_jobs = n_jobs)
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric=metrics_dict[eval_metric]['lgb_metric_name'],
                    verbose=verbose, early_stopping_rounds=early_stopping_rounds)
            
            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test, num_iteration=model.best_iteration_)[:, 1]
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=n_estimators, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=verbose, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid)
            print(f'Fold {fold_n}. {eval_metric}: {score:.4f}.')
            print('')
            
            y_pred = model.predict_proba(X_test)
        
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=n_estimators, eval_metric=metrics_dict[eval_metric]['catboost_metric_name'], **params,
                                      loss_function='Logloss')
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if averaging == 'usual':
            
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
            
            prediction += y_pred.reshape(-1, 1)

        elif averaging == 'rank':
                                  
            oof[valid_index] = y_pred_valid.reshape(-1, 1)
            scores.append(metrics_dict[eval_metric]['sklearn_scoring_function'](y_valid, y_pred_valid))
                                  
            prediction += pd.Series(y_pred).rank().values.reshape(-1, 1)        
        
        if model_type == 'lgb' and plot_feature_importance:
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_splits
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    result_dict['oof'] = oof
    result_dict['prediction'] = prediction
    result_dict['scores'] = scores
    
    if model_type == 'lgb':
        if plot_feature_importance:
            feature_importance["importance"] /= n_splits
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            bottom_cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[-50:].index 
            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12))
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
            plt.title('LGB Features (avg over folds)')
            
            result_dict['feature_importance'] = feature_importance
            result_dict['top_columns'] = cols
            result_dict['bottom_columns'] = bottom_cols
        
    return result_dict


In [52]:
data = pd.concat(( train, test ))
data = data.iloc[ np.random.permutation(len( data )) ]
data.reset_index( drop = True, inplace = True )
train = data

In [53]:
cat_cols = ['MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO',
'SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU'] #'COR_KEY_PROD_HLD_NBR',
question_cols = ['REG_CPT','CUR_YEAR_MID_BUS_INC']
large_cale_cols = ['ICO_CUR_MON_ACM_TRX_AMT',
'COUNTER_CUR_YEAR_CNT_AMT',
'PUB_TO_PRV_TRX_AMT_CUR_YEAR',
'MON_12_EXT_SAM_TRSF_IN_AMT',
'MON_12_EXT_SAM_TRSF_OUT_AMT',
'MON_12_EXT_SAM_AMT',
'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT',
'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT',
'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL',
'CUR_MON_VAL_VLD_CUST_NED_HLD_YEAR_DAY_AVG',
'LAST_12_MON_COR_DPS_DAY_AVG_BAL',
'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV',
'LAST_12_MON_MON_AVG_TRX_AMT_NAV'
]
continue_cols = []

# 删除特征
need_drop_col = ['MON_12_CUST_CNT_PTY_ID']
train.drop(need_drop_col, axis=1)
test.drop(need_drop_col, axis=1)


for col in train.columns:
    if col not in ['CUST_UID','LABEL']+cat_cols+question_cols+large_cale_cols:
        continue_cols.append(col)
# 对类别特征labelencoder
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

n_fold = 5
#folds = TimeSeriesSplit(n_splits=n_fold)
folds = KFold(n_splits=5)

# single_onehot = ['MON_12_CUST_CNT_PTY_ID','AI_STAR_SCO','WTHR_OPN_ONL_ICO',
# 'SHH_BCK','LGP_HLD_CARD_LVL','NB_CTC_HLD_IDV_AIO_CARD_SITU']
# for col in single_onehot:
#     try:
#         print("%s_one_hot"%col)
#         train[col + 'one_hot'] = OneHotEncoder().fit_transform(train[col].values.reshape(-1, 1))
#         test[col + 'one_hot'] = OneHotEncoder().fit_transform(train[col].values.reshape(-1, 1))
#     except:
#         print("%s is not valid!"%col)

# train.head()
# print(len(list(train.columns)))
# features = [['MON_12_EXT_SAM_TRSF_IN_AMT','MON_12_EXT_SAM_TRSF_OUT_AMT'],['MON_12_EXT_SAM_NM_TRSF_OUT_CNT','MON_12_EXT_SAM_AMT'],['CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT','CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT']
# ,['CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL','CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR']]
# def get_row_fea(df,features):
#     for fea in features:
#         # df[f'{fea[0]}_{fea[1]}_std'] = df[fea].std(1)
#         # df[f'{fea[0]}_{fea[1]}_max'] = df[fea].max(1)
#         # df[f'{fea[0]}_{fea[1]}_min'] = df[fea].min(1)
#         # df[f'{fea[0]}_{fea[1]}_sub'] = df[fea[0]] - df[fea[1]]

#         df.loc[df[fea[0]] <= df[fea[1]], f'{fea[0]}_{fea[1]}_mark'] = 0
#         df.loc[df[fea[0]] > df[fea[1]], f'{fea[0]}_{fea[1]}_mark'] = 1
#     return df
# train = get_row_fea(train,features)  
# test = get_row_fea(test,features)  
# print(len(list(train.columns)))



In [54]:
#continue_cols+ ['CUST_UID','LABEL']+cat_cols+question_cols+large_cale_cols
X = train.sort_values('CUST_UID').drop(['TARGET','LABEL', 'CUST_UID'], axis=1)
y = train.sort_values('CUST_UID')['TARGET']
X_test = test.drop(['CUST_UID'], axis=1)

def clean_inf_nan(df):
    return df.replace([np.inf, -np.inf], np.nan)   

# Cleaning infinite values to NaN
X = clean_inf_nan(X)
X_test = clean_inf_nan(X_test )


print(len(X.columns))
print(X.shape)

49
(52000, 49)


In [55]:
print('fea lens:',len(X.columns))
averag = 'usual'#'rank'#
model_name = 'lgb'
params = {'num_leaves': 63,
          'min_child_samples': 79,
          'objective': 'binary',
          'max_depth': 7,
          'learning_rate': 0.08,#0.03
          "boosting_type": "gbdt",
          "subsample_freq": 3,
          "subsample": 0.8,
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3,
          'reg_lambda': 0.3,
          'colsample_bytree': 0.8#,
          #'categorical_feature': cat_cols
         }

result_dict_lgb = train_model_classification(X=X, X_test=X_test, y=y, params=params, folds=folds, model_type=model_name, eval_metric='auc', plot_feature_importance=True,
                                                      verbose=200, early_stopping_rounds=200, n_estimators=5000, averaging=averag, n_jobs=-1)

fea lens: 49
Fold 1 started at Tue May 10 15:13:06 2022
[200]	training's auc: 0.887195	training's auc: 0.887195	valid_1's auc: 0.497192	valid_1's auc: 0.497193
Fold 2 started at Tue May 10 15:14:30 2022
[200]	training's auc: 0.881271	training's auc: 0.881271	valid_1's auc: 0.49773	valid_1's auc: 0.49773
[400]	training's auc: 0.952976	training's auc: 0.952976	valid_1's auc: 0.504591	valid_1's auc: 0.504591


KeyboardInterrupt: 

In [41]:
# meta_train = pd.DataFrame()
# meta_test = pd.DataFrame()
# meta_train['predictions'] = pd.Series(result_dict_lgb['oof'].reshape(-1))#.rank()
#seperate train rows which have been misclassified as test and use them as validation
train2 = train.sort_values('CUST_UID')
train2["predictions"] = pd.Series(result_dict_lgb['oof'].reshape(-1))
predictions_argsort = train2['predictions'].argsort()
#print(predictions_argsort)
train_sorted = train2.iloc[predictions_argsort]
# print(train_sorted.tail())
#select only trains set because we need to find train rows which have been misclassified as test set and use them for validation
train_sorted = train_sorted.loc[train_sorted.TARGET == 0]

#Why did I chose 0.7 as thereshold? just a hunch, but you should try different thresholds i.e 0.6, 0.8 and see the difference in validation score and please report back. :) 
train_as_test = train_sorted.loc[train_sorted.predictions > 0.7]
print(train_as_test.head())
train_as_test.to_csv('adversarial_set_data.csv', index=False)
#save the indices of the misclassified train rows to use as validation set
adversarial_set_ids = train_as_test.index.values
adversarial_set = pd.DataFrame(adversarial_set_ids, columns=['adversial_set_ids'])
#save adversarial set index
adversarial_set.to_csv('adversarial_set_ids.csv', index=False)
print(adversarial_set.shape)
adversarial_set.head()

                               CUST_UID  LABEL  AGN_CNT_RCT_12_MON  \
33735  c07e9acedf7f4d67bbfabd420bd4a957    1.0                 NaN   
30387  e56a8aab364042e3b6b2ee339fe51132    0.0                 NaN   
23988  0d33bf93738b44a3af8b64fa482059b7    0.0                 NaN   
21660  55ed95b6c7f04e958627ec3661093e97    0.0                 NaN   
25645  781d328b5c2e43a59bf52de46db72b89    1.0              1322.0   

       ICO_CUR_MON_ACM_TRX_TM  NB_RCT_3_MON_LGN_TMS_AGV  \
33735                    62.0                       NaN   
30387                   122.0       612.000000000000000   
23988                   402.0      1392.000000000000000   
21660                     NaN                       NaN   
25645                   472.0       255.300000000000011   

            AGN_CUR_YEAR_AMT  AGN_CUR_YEAR_WAG_AMT  AGN_AGR_LATEST_AGN_AMT  \
33735                    NaN                   NaN                     NaN   
30387                    NaN                   NaN                  

,adversial_set_ids
0,33735
1,30387
2,23988
3,21660
4,25645
